# Stokes Eigenvalue Solver with NGSolve

In [2]:
from ngsolve import *
from ngsolve.webgui import Draw

import scipy.sparse as sp
from scipy.sparse.linalg import eigs, eigsh, norm
import numpy as np

## 1. Unit Square

In [9]:
def bilinearform_to_scipy(a):
    # sparse scipy matrix
    rows,cols,vals = a.mat.COO()
    A = sp.csr_matrix((vals,(rows,cols)))
    return A
def compute_eigenvalues_bilinearform(a, m, num_eigenvalues, shift=0.0):
    # sparse scipy matrices
    Afull = bilinearform_to_scipy(a)
    Mfull = bilinearform_to_scipy(m)
    
    fd = np.nonzero(a.space.FreeDofs())[0]
    # velocity_space = a.space.components[0]
    # pressure_fixed_dof = velocity_space.ndof
    # fd = np.setdiff1d(fd, [pressure_fixed_dof])
    A = Afull[fd,:][:,fd]
    M = Mfull[fd,:][:,fd]
    print("Computing eigenvalues...")
    fes = a.space
    lams_SM, ui = eigs(A = A, k = num_eigenvalues, M=M, tol=1e-12, return_eigenvectors=True, sigma=shift, which='LM', OPpart='r')
    print("Computed eigenvalues (shift-and-invert):", lams_SM)
    ui_temp = np.zeros((fes.ndof,num_eigenvalues), dtype=np.float64)
    for j in range(num_eigenvalues):
        ui_temp[fd,j] = ui[:,j].real
    return ui_temp, lams_SM               
def Stokes_bilinearforms(mesh, dirichletbnd=[], avgpressure=False):
    V = VectorH1(mesh, order=2, dirichlet = dirichletbnd)  
    Q = H1(mesh, order=1)  
    if avgpressure:
        N = NumberSpace(mesh)
        X= V*Q*N
        (u,p,lam), (v,q,mu) = X.TnT()
        epsilon=1e-12
        a = BilinearForm(X)
        a += InnerProduct(Grad(u),Grad(v))*dx + div(u)*q*dx + div(v)*p*dx +epsilon*p*q*dx + lam*q*dx + mu*p*dx
        a.Assemble()
    else:
        X = V*Q
        (u,p), (v,q) = X.TnT()
        epsilon=1e-12
        a = BilinearForm(X)
        a += InnerProduct(Grad(u),Grad(v))*dx + div(u)*q*dx + div(v)*p*dx +epsilon*p*q*dx
        a.Assemble()
    m = BilinearForm(X)
    m += u*v*dx 
    m.Assemble()       
    return a, m


In [21]:
unit_mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
a,m = Stokes_bilinearforms(unit_mesh, dirichletbnd="left|bottom|right|top")
ui_unitsq_scipy, lams_SM = compute_eigenvalues_bilinearform(a, m, num_eigenvalues=20)


Computing eigenvalues...
Computed eigenvalues (shift-and-invert): [ 52.36254439+0.j  92.2054846 +0.j  92.20056076+0.j 128.42739208+0.j
 154.45735762+0.j 167.43596177+0.j 190.1968611 +0.j 190.22045907+0.j
 247.69599783+0.j 247.51883378+0.j 247.58585607+0.j 270.7699414 +0.j
 280.91191647+0.j 329.60201303+0.j 329.58249926+0.j 352.73822   +0.j
 366.52133115+0.j 384.53472349+0.j 384.72666996+0.j 409.37609531+0.j]


In [22]:
num = 20
gf_stokes = GridFunction(a.space, multidim=num)
for j in range(0,num):
    gf_stokes.vecs[j].FV().NumPy()[:] = ui_unitsq_scipy[:,j].real

In [23]:
Draw(gf_stokes.components[0], unit_mesh, 'eigenmodes_velocity')
Draw(gf_stokes.components[1], unit_mesh, 'eigenmodes_pressure')

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## 2. U mesh

In [4]:
Umesh_refinada = Mesh("U_mesh_refinada.vol")

In [ ]:
a,m = Stokes_bilinearforms(Umesh_refinada, dirichletbnd="wall")
ui_diff_scipy, lams_SM = compute_eigenvalues_bilinearform(a, m, num_eigenvalues=20,shift=1000)

Computing eigenvalues...


In [ ]:
num = 20
gf_stokes = GridFunction(a.space, multidim=num)
for j in range(0,num):
    gf_stokes.vecs[j].FV().NumPy()[:] = ui_diff_scipy[:,j].real

In [ ]:
Draw(gf_stokes.components[0], Umesh_refinada, 'eigenmodes_velocity')

# 3. Arteria 2d

In [10]:
arteria2d_mesh = Mesh("arteria_mesh.vol")
# Draw(arteria2d_mesh)

In [11]:
a,m = Stokes_bilinearforms(arteria2d_mesh, dirichletbnd="wall")
ui_diff_scipy, lams_SM = compute_eigenvalues_bilinearform(a, m, num_eigenvalues=20,shift=1000)


Computing eigenvalues...
Computed eigenvalues (shift-and-invert): [1350.15692102+0.j 1851.61846105+0.j 2083.31891011+0.j 2352.80227429+0.j
 2428.43268704+0.j 2506.2614398 +0.j 2529.24696152+0.j 2518.98344245+0.j
 2623.38855957+0.j 2760.62275645+0.j 2812.21728114+0.j 2886.39380962+0.j
 2980.43248217+0.j 3004.18063861+0.j 3074.10360571+0.j 3129.33823936+0.j
 3184.48642457+0.j 3210.4870617 +0.j 3269.49306253+0.j 3343.7123168 +0.j]


In [12]:
num = 20
gf_stokes = GridFunction(a.space, multidim=num)
for j in range(0,num):
    gf_stokes.vecs[j].FV().NumPy()[:] = ui_diff_scipy[:,j].real

In [13]:
Draw(gf_stokes.components[1], arteria2d_mesh, 'eigenmodes_velocity')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene